In [ ]:
#30/7/24 Creates a K fold KNN model for each file in a folder provided they are Fingerprints
import gc
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '999'
import pandas as pd
import numpy as np
import math
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import nnmetrics
from Function_Repo import setbalance
import math
import tensorflow as tf
from tensorflow import keras
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Fingerprints/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df.drop(columns=['SMILES'], inplace=True)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    threshlist = []
    finalresults = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        
        
        keras.utils.set_random_seed(seed)
        tf.config.experimental.enable_op_determinism()
        length = len(xtrain.columns)
        sampcount = len(ytrain)
        temp = length
        # Encoder Creation
        enc_in = keras.Input(shape=(length, 1))
        x = keras.layers.Flatten()(enc_in)
        # Additional encoder layers
        
        layer1 = math.sqrt(3*sampcount) + 2*(math.sqrt(sampcount/(3)))
        x = keras.layers.Dense(layer1, activation="relu")(x)
        
        layer2 = math.sqrt(sampcount / 3)
        x = keras.layers.Dense(layer2, activation="relu")(x)
        
        enc_out = keras.layers.Dense(1, activation="sigmoid")(x)
        autoencoder = keras.Model(enc_in, enc_out, name='autoencoder')
        # Optimiser
        
        optimise = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

        autoencoder.compile(optimise, loss='binary_crossentropy')
        #autoencoder.summary()
        
        ytrain = np.array(ytrain)
        epochs = 500
        batchsize = 16
        losshistory = []

        minloss = 1
        bestepoch = 0
        for epoch in range(epochs):
            verb = 0
            logs = autoencoder.fit(
            xtrain,
            ytrain,
            verbose = verb,
            epochs=1,
            batch_size=batchsize),
            validation_split=0.2,

            #autoencoder.save(str(f"models/AE-{epoch+1}.keras"))
            loss = logs[0].history['loss'][0]
            losshistory.append(loss)
            if loss < minloss:
                minloss = loss
                bestepoch = epoch

        model_list.append(autoencoder)
    
        preds = autoencoder.predict(xtest)

        epochcount = range(epochs)

        acclist = []
        thresholdlist = []
        maxval = 0
        bestthresh = 0
        for threshold in np.arange(0, 1, 0.01):
            newval = nnmetrics(preds, ytest, threshold)[12]
            acclist.append(newval)
            thresholdlist.append(threshold)
            if newval > maxval:
                maxval = newval
                bestthresh = threshold
        threshlist.append(bestthresh)
        testdat.append(maxval)
        y_values = acclist
        x_values = thresholdlist
        preds = autoencoder.predict(xvalid)
        results = nnmetrics(preds, yvalid, bestthresh)
        validlist.append(results)
        print('for fold', split + 1, 'optimal test mcc of', maxval, 'validation mcc of', results[12])

        iteration = iteration + 1

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        

    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []

    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        fp = np.asarray(fp).astype('float32')
        #appends each model's prediction to a list
        predlist = []
        for model in model_list:
            preds = model.predict(fp, verbose = 0)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        loops = 0
        for modelpreds in predlist:
            if modelpreds < threshlist[loops]:
                zercount = zercount + 1
        loops = loops + 1

        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)

    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    finalresults.append(results[12])
    del autoencoder
    gc.collect
    
    sets += 1

For dataset NR-AR-LBD_Avafp.csv
2/2 [==============================] - 0s 792us/step
for fold 1 optimal test mcc of 0.6871657754010695 validation mcc of 0.48122447405980645
2/2 [==============================] - 0s 732us/step
for fold 2 optimal test mcc of 0.5786882152809495 validation mcc of 0.46046933104270676
2/2 [==============================] - 0s 707us/step
for fold 3 optimal test mcc of 0.610170215847752 validation mcc of 0.4308571590681121
2/2 [==============================] - 0s 789us/step
for fold 4 optimal test mcc of 0.7378647873726218 validation mcc of 0.4947753344488257
2/2 [==============================] - 0s 758us/step
for fold 5 optimal test mcc of 0.5693438283463509 validation mcc of 0.5786882152809495

validation metrics of:
positives in data 47
negatives in data 49
net accuracy = 0.7604166666666666
mcc = 0.5248138373964939
For dataset SR-ATAD5_Morganfp.csv
2/2 [==============================] - 0s 838us/step
for fold 1 optimal test mcc of 0.6431758808251548 valid

For dataset Hepatotoxicity_Morganfp.csv
8/8 [==============================] - 0s 489us/step
for fold 1 optimal test mcc of 0.45418633434205224 validation mcc of 0.35032448485149137
8/8 [==============================] - 0s 565us/step
for fold 2 optimal test mcc of 0.48374383424080136 validation mcc of 0.35441354799467134
8/8 [==============================] - 0s 498us/step
for fold 3 optimal test mcc of 0.45268663757863004 validation mcc of 0.3872723137560331
8/8 [==============================] - 0s 673us/step
for fold 4 optimal test mcc of 0.3436856069281347 validation mcc of 0.3781359338351391
8/8 [==============================] - 0s 507us/step
for fold 5 optimal test mcc of 0.4294542524608862 validation mcc of 0.42290140816864585

validation metrics of:
positives in data 293
negatives in data 285
net accuracy = 0.7024221453287197
mcc = 0.41026516352225684
For dataset Cardiotoxicity-5_Morganfp.csv
4/4 [==============================] - 0s 587us/step
for fold 1 optimal test mcc of 

6/6 [==============================] - 0s 437us/step
for fold 1 optimal test mcc of 0.38606481356256656 validation mcc of 0.2594286294661387
6/6 [==============================] - 0s 507us/step
for fold 2 optimal test mcc of 0.3242318758419489 validation mcc of 0.3147876717164867
6/6 [==============================] - 0s 467us/step
for fold 3 optimal test mcc of 0.2100906102931654 validation mcc of 0.17014497764136072
6/6 [==============================] - 0s 484us/step
for fold 4 optimal test mcc of 0.3246861882798744 validation mcc of 0.3324967857801223
6/6 [==============================] - 0s 425us/step
for fold 5 optimal test mcc of 0.36699012778230794 validation mcc of 0.1999417458294882

validation metrics of:
positives in data 181
negatives in data 243
net accuracy = 0.6768867924528302
mcc = 0.3257074870725125
For dataset Hepatotoxicity_Avafp.csv
8/8 [==============================] - 0s 423us/step
for fold 1 optimal test mcc of 0.5001403571054212 validation mcc of 0.3698967791

2/2 [==============================] - 0s 1ms/step
for fold 3 optimal test mcc of 0.3763493004997114 validation mcc of 0.5379519296838619
2/2 [==============================] - 0s 1ms/step
for fold 4 optimal test mcc of 0.42420801505049993 validation mcc of 0.40982808233692364
2/2 [==============================] - 0s 1ms/step
for fold 5 optimal test mcc of 0.6349812749303677 validation mcc of 0.3802528761961016

validation metrics of:
positives in data 70
negatives in data 76
net accuracy = 0.6301369863013698
mcc = 0.25828342543771304
For dataset Reproductive_Toxicity_MACCSfp.csv
1/1 [==============================] - 0s 9ms/step
for fold 1 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 2 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 3 optimal test mcc of 0 validation mcc of 0
1/1 [==============================] - 0s 9ms/step
for fold 4 optimal test mcc of 0 validation mcc o

2/2 [==============================] - 0s 912us/step
for fold 4 optimal test mcc of 0.37259615384615385 validation mcc of 0.4385566936919029
2/2 [==============================] - 0s 787us/step
for fold 5 optimal test mcc of 0.4693339858180978 validation mcc of 0.36886386424569245

validation metrics of:
positives in data 70
negatives in data 76
net accuracy = 0.7123287671232876
mcc = 0.43142098500564113
For dataset NR-AR_Morganfp.csv
2/2 [==============================] - 0s 818us/step
for fold 1 optimal test mcc of 0.6002099318605758 validation mcc of 0.5424327604368505
2/2 [==============================] - 0s 956us/step
for fold 2 optimal test mcc of 0.6564322015646866 validation mcc of 0.3611575592573076
2/2 [==============================] - 0s 865us/step
for fold 3 optimal test mcc of 0.3595462477533374 validation mcc of 0.6082291325648282
2/2 [==============================] - 0s 826us/step
for fold 4 optimal test mcc of 0.6371766120160759 validation mcc of 0.38697637169562726


...

For dataset Respiratory_Toxicity_TTorsionfp.csv
4/4 [==============================] - 0s 503us/step
for fold 1 optimal test mcc of 0.32550457133142946 validation mcc of 0.3735632183908046
4/4 [==============================] - 0s 488us/step
for fold 2 optimal test mcc of 0.4216001839710295 validation mcc of 0.3086577323779664
4/4 [==============================] - 0s 616us/step
for fold 3 optimal test mcc of 0.4477466273881998 validation mcc of 0.5222329678670935
4/4 [==============================] - 0s 498us/step
for fold 4 optimal test mcc of 0.5294283510409393 validation mcc of 0.463423748665829
4/4 [==============================] - 0s 460us/step
for fold 5 optimal test mcc of 0.5439299103815997 validation mcc of 0.22359804471289746

validation metrics of:
positives in data 164
negatives in data 106
net accuracy = 0.725925925925926
mcc = 0.4489881263292412
For dataset NR-AR-LBD_TTorsionfp.csv
2/2 [==============================] - 0s 737us/step
for fold 1 optimal test mcc of 0.63

4/4 [==============================] - 0s 513us/step
for fold 2 optimal test mcc of 0.45345112042656843 validation mcc of 0.38194387509081823
4/4 [==============================] - 0s 499us/step
for fold 3 optimal test mcc of 0.3508600569648358 validation mcc of 0.41704373879893003
4/4 [==============================] - 0s 625us/step
for fold 4 optimal test mcc of 0.3708550575378807 validation mcc of 0.3684538969338882
4/4 [==============================] - 0s 530us/step
for fold 5 optimal test mcc of 0.5433697176706682 validation mcc of 0.5082598904572859

validation metrics of:
positives in data 190
negatives in data 120
net accuracy = 0.7193548387096774
mcc = 0.3886344286147219
For dataset SR-p53_Avafp.csv
3/3 [==============================] - 0s 533us/step
for fold 1 optimal test mcc of 0.49760883235118014 validation mcc of 0.23853989813242785
3/3 [==============================] - 0s 564us/step
for fold 2 optimal test mcc of 0.4664341761115955 validation mcc of 0.4386595048236857

6/6 [==============================] - 0s 410us/step
for fold 3 optimal test mcc of 0.5784495923684765 validation mcc of 0.5379114627956251
6/6 [==============================] - 0s 428us/step
for fold 4 optimal test mcc of 0.6439687475658502 validation mcc of 0.4928170855147789
6/6 [==============================] - 0s 450us/step
for fold 5 optimal test mcc of 0.666390340326285 validation mcc of 0.6094443579851524

validation metrics of:
positives in data 179
negatives in data 243
net accuracy = 0.7843601895734598
mcc = 0.5611491894811158
For dataset Cardiotoxicity-1_TTorsionfp.csv
4/4 [==============================] - 0s 508us/step
for fold 1 optimal test mcc of 0.42826724500232755 validation mcc of 0.4394335853995134
4/4 [==============================] - 0s 563us/step
for fold 2 optimal test mcc of 0.5392248643357401 validation mcc of 0.4261285602483525
4/4 [==============================] - 0s 499us/step
for fold 3 optimal test mcc of 0.497012816875114 validation mcc of 0.4670926

2/2 [==============================] - 0s 768us/step
for fold 3 optimal test mcc of 0.38517524110239 validation mcc of 0.5530797881778003
2/2 [==============================] - 0s 775us/step
for fold 4 optimal test mcc of 0.4474593028095495 validation mcc of 0.5272919760785556
2/2 [==============================] - 0s 776us/step
for fold 5 optimal test mcc of 0.4776988814092718 validation mcc of 0.36140316116210053

validation metrics of:
positives in data 73
negatives in data 79
net accuracy = 0.631578947368421
mcc = 0.2766010042182998
For dataset SR-MMP_TTorsionfp.csv
6/6 [==============================] - 0s 498us/step
for fold 1 optimal test mcc of 0.599611180673214 validation mcc of 0.4790304739342625
6/6 [==============================] - 0s 433us/step
for fold 2 optimal test mcc of 0.5121408168242175 validation mcc of 0.38197486314731427
6/6 [==============================] - 0s 519us/step
for fold 3 optimal test mcc of 0.33605688291099356 validation mcc of 0.4449680007856228
6/

2/2 [==============================] - 0s 805us/step
for fold 4 optimal test mcc of 0.38432209954297986 validation mcc of 0.1543033499620919
2/2 [==============================] - 0s 786us/step
for fold 5 optimal test mcc of 0.6360826525958092 validation mcc of 0.24942329796316198

validation metrics of:
positives in data 85
negatives in data 53
net accuracy = 0.6594202898550725
mcc = 0.31771061197261063
For dataset SR-ATAD5_Avafp.csv
2/2 [==============================] - 0s 708us/step
for fold 1 optimal test mcc of 0.5469992182776682 validation mcc of 0.31622776601683794
2/2 [==============================] - 0s 874us/step
for fold 2 optimal test mcc of 0.5085118434825329 validation mcc of 0.27060403659669535
2/2 [==============================] - 0s 751us/step
for fold 3 optimal test mcc of 0.5196667888450939 validation mcc of 0.42008402520840293
2/2 [==============================] - 0s 714us/step
for fold 4 optimal test mcc of 0.5333795796988011 validation mcc of 0.257513101312302

In [3]:
temp = pd.DataFrame(finalresults, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Data/Model_Outputs/Fingerprint/NeuralNet_Fingerprint_Results')
resultframe

,Dataset,MCC
0,CarcinogenicityMorganfp.csv,0.0
1,morganfp_SR-p53.csv,NaN
2,morganfp_NR-AhR.csv,NaN
3,morganfp_NR-ER-LBD.csv,NaN
4,mutagenTTorsionfp.csv,NaN
5,morganfp_NR-PPAR-gamma.csv,NaN
6,Respiratory_ToxicityMorganfp.csv,NaN
7,mutagenMACCSfp.csv,NaN
8,Reproductive_ToxicityTTorsionfp.csv,NaN
9,CarcinogenicityAvafp.csv,NaN


In [4]:
finalresults

[0]